# 1. 구글 드라이브 연결 및 필요한 라이브러리 설치

## 1-1. 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1-2. 경로 지정 및 필요한 라이브러리 설치

In [ ]:
path = '/content/drive/MyDrive/연세_수업자료/3학기/기계학습및프로그래밍/데이터/학습_및_검증/비대면진료_환자_학습/'
!pip install tqdm
import os, json
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
tqdm.pandas(desc="Checking files")

# with open(path + '/transcription/PC_0004/PC_0004-8233-01-03-M-04-A.json', 'r') as f:
#     data = json.load(f)
# data

In [ ]:
# data['전사정보']['LabelText']

# 2. Raw data 만들기

## 2-1. transcription 기준으로 데이터프레임 만들기

In [ ]:
result = []
for foldername in tqdm(os.listdir(path + '/transcription')):
    for filename in os.listdir(path + '/transcription/' + foldername):
        with open(path + '/transcription/' + foldername + '/' + filename, 'r') as f:
            data = json.load(f)

        file_prefix = filename.split('-')[0]
        audio_file_path = 'audio/{}/{}.wav'.format(foldername, filename[:-5])
        text_file_path = 'transcription/{}/{}.json'.format(foldername, filename[:-5])
        labeltext = data['전사정보']['LabelText']
        gender = data['화자정보']['Gender']
        age = data['화자정보']['Age']
        region = data['화자정보']['Region']
        dialect = data['화자정보']['Dialect']
        samplingrate = data['음성정보']['SamplingRate']
        qualitystatus = data['기타정보']['QualityStatus']

        result.append([filename[:-5],
                       audio_file_path,
                       text_file_path,
                       labeltext,
                       gender,
                       age,
                       region,
                       dialect,
                       samplingrate,
                       qualitystatus])

  0%|          | 0/885 [00:00<?, ?it/s]

In [ ]:
# 데이터프레임 만들기
df = pd.DataFrame(result, columns=['filename', 'audio_file_path', 'text_file_path', 'labeltext', 'gender', 'age', 'region', 'dialect', 'samplingrate', 'qualitystatus'])

In [ ]:
df

,filename,audio_file_path,text_file_path,labeltext,gender,age,region,dialect,samplingrate,qualitystatus
0,PD_0364-15298-01-03-F-05-B,audio/PD_0364/PD_0364-15298-01-03-F-05-B.wav,transcription/PD_0364/PD_0364-15298-01-03-F-05...,자정 전까지 아무거나 먹어도 돼요?,Female,30~39,대전/세종/충청/강원,충청,48000,Good
1,PD_0364-15398-01-03-F-05-B,audio/PD_0364/PD_0364-15398-01-03-F-05-B.wav,transcription/PD_0364/PD_0364-15398-01-03-F-05...,음. 아니요.,Female,30~39,대전/세종/충청/강원,충청,48000,Good
2,PD_0364-15490-01-03-F-05-B,audio/PD_0364/PD_0364-15490-01-03-F-05-B.wav,transcription/PD_0364/PD_0364-15490-01-03-F-05...,소변이 살짝 탁한 색이었어요.,Female,30~39,대전/세종/충청/강원,충청,48000,Good
3,PD_0364-15561-01-03-F-05-B,audio/PD_0364/PD_0364-15561-01-03-F-05-B.wav,transcription/PD_0364/PD_0364-15561-01-03-F-05...,오늘 점심으로 돈가스랑 마셨어요. 한 시 정도에요.,Female,30~39,대전/세종/충청/강원,충청,48000,Good
4,PD_0364-15396-01-03-F-05-B,audio/PD_0364/PD_0364-15396-01-03-F-05-B.wav,transcription/PD_0364/PD_0364-15396-01-03-F-05...,다이어트하느라 변비가 생겨서 요즘 화장실을 잘 못 가요.,Female,30~39,대전/세종/충청/강원,충청,48000,Good
...,...,...,...,...,...,...,...,...,...,...
309397,PF_0338-23233-01-03-M-04-B,audio/PF_0338/PF_0338-23233-01-03-M-04-B.wav,transcription/PF_0338/PF_0338-23233-01-03-M-04...,어느 쪽으로 눕혀야 되나요?,Male,20~29,대전/세종/충청/강원,충청,48000,Good
309398,PF_0338-23350-01-03-M-04-B,audio/PF_0338/PF_0338-23350-01-03-M-04-B.wav,transcription/PF_0338/PF_0338-23350-01-03-M-04...,몸무게가 오십 조금 넘으실 거예요. 키는 모르겠네요.,Male,20~29,대전/세종/충청/강원,충청,48000,Good
309399,PF_0338-23450-01-03-M-04-B,audio/PF_0338/PF_0338-23450-01-03-M-04-B.wav,transcription/PF_0338/PF_0338-23450-01-03-M-04...,자리를 지키고 있을 수가 없어서요. 꼭 문자 메시지 보내주세요.,Male,20~29,대전/세종/충청/강원,충청,48000,Good
309400,PF_0338-23563-01-03-M-04-B,audio/PF_0338/PF_0338-23563-01-03-M-04-B.wav,transcription/PF_0338/PF_0338-23563-01-03-M-04...,저는 다인실로 가지 않고 이인실에 쭉 있을 겁니다.,Male,20~29,대전/세종/충청/강원,충청,48000,Good


## 2-2. 폴더에 없는 파일 없앤 raw DataFrame 만들기

In [ ]:
# df = df.drop(columns=['file_exists'])

In [ ]:
# 파일 존재 여부를 확인하는 함수
def file_exists(row):
    audio_exists = os.path.exists(path + row['audio_file_path'])
    text_exists = os.path.exists(path + row['text_file_path'])
    return audio_exists, text_exists

In [ ]:
# 데이터프레임에 파일 존재 여부를 확인하는 열 추가
results = df.progress_apply(file_exists, axis=1, result_type='expand')
df[['audio_exists', 'text_exists']] = results

Checking files:   0%|          | 0/309402 [00:00<?, ?it/s]

In [ ]:
df

,filename,audio_file_path,text_file_path,labeltext,gender,age,region,dialect,samplingrate,qualitystatus,audio_exists,text_exists
0,PD_0364-15298-01-03-F-05-B,audio/PD_0364/PD_0364-15298-01-03-F-05-B.wav,transcription/PD_0364/PD_0364-15298-01-03-F-05...,자정 전까지 아무거나 먹어도 돼요?,Female,30~39,대전/세종/충청/강원,충청,48000,Good,True,True
1,PD_0364-15398-01-03-F-05-B,audio/PD_0364/PD_0364-15398-01-03-F-05-B.wav,transcription/PD_0364/PD_0364-15398-01-03-F-05...,음. 아니요.,Female,30~39,대전/세종/충청/강원,충청,48000,Good,True,True
2,PD_0364-15490-01-03-F-05-B,audio/PD_0364/PD_0364-15490-01-03-F-05-B.wav,transcription/PD_0364/PD_0364-15490-01-03-F-05...,소변이 살짝 탁한 색이었어요.,Female,30~39,대전/세종/충청/강원,충청,48000,Good,True,True
3,PD_0364-15561-01-03-F-05-B,audio/PD_0364/PD_0364-15561-01-03-F-05-B.wav,transcription/PD_0364/PD_0364-15561-01-03-F-05...,오늘 점심으로 돈가스랑 마셨어요. 한 시 정도에요.,Female,30~39,대전/세종/충청/강원,충청,48000,Good,True,True
4,PD_0364-15396-01-03-F-05-B,audio/PD_0364/PD_0364-15396-01-03-F-05-B.wav,transcription/PD_0364/PD_0364-15396-01-03-F-05...,다이어트하느라 변비가 생겨서 요즘 화장실을 잘 못 가요.,Female,30~39,대전/세종/충청/강원,충청,48000,Good,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
309397,PF_0338-23233-01-03-M-04-B,audio/PF_0338/PF_0338-23233-01-03-M-04-B.wav,transcription/PF_0338/PF_0338-23233-01-03-M-04...,어느 쪽으로 눕혀야 되나요?,Male,20~29,대전/세종/충청/강원,충청,48000,Good,True,True
309398,PF_0338-23350-01-03-M-04-B,audio/PF_0338/PF_0338-23350-01-03-M-04-B.wav,transcription/PF_0338/PF_0338-23350-01-03-M-04...,몸무게가 오십 조금 넘으실 거예요. 키는 모르겠네요.,Male,20~29,대전/세종/충청/강원,충청,48000,Good,True,True
309399,PF_0338-23450-01-03-M-04-B,audio/PF_0338/PF_0338-23450-01-03-M-04-B.wav,transcription/PF_0338/PF_0338-23450-01-03-M-04...,자리를 지키고 있을 수가 없어서요. 꼭 문자 메시지 보내주세요.,Male,20~29,대전/세종/충청/강원,충청,48000,Good,True,True
309400,PF_0338-23563-01-03-M-04-B,audio/PF_0338/PF_0338-23563-01-03-M-04-B.wav,transcription/PF_0338/PF_0338-23563-01-03-M-04...,저는 다인실로 가지 않고 이인실에 쭉 있을 겁니다.,Male,20~29,대전/세종/충청/강원,충청,48000,Good,True,True


In [ ]:
# 파일이 존재하지 않는 행 제거 후 불필요한 열 삭제
filtered_df = df[(df['audio_exists'] == True) & (df['text_exists'] == True)]
filtered_df = filtered_df.drop(columns=['audio_exists'])
filtered_df = filtered_df.drop(columns=['text_exists'])

In [ ]:
# 결과 출력
filtered_df

,filename,audio_file_path,text_file_path,labeltext,gender,age,region,dialect,samplingrate,qualitystatus
0,PD_0364-15298-01-03-F-05-B,audio/PD_0364/PD_0364-15298-01-03-F-05-B.wav,transcription/PD_0364/PD_0364-15298-01-03-F-05...,자정 전까지 아무거나 먹어도 돼요?,Female,30~39,대전/세종/충청/강원,충청,48000,Good
1,PD_0364-15398-01-03-F-05-B,audio/PD_0364/PD_0364-15398-01-03-F-05-B.wav,transcription/PD_0364/PD_0364-15398-01-03-F-05...,음. 아니요.,Female,30~39,대전/세종/충청/강원,충청,48000,Good
2,PD_0364-15490-01-03-F-05-B,audio/PD_0364/PD_0364-15490-01-03-F-05-B.wav,transcription/PD_0364/PD_0364-15490-01-03-F-05...,소변이 살짝 탁한 색이었어요.,Female,30~39,대전/세종/충청/강원,충청,48000,Good
3,PD_0364-15561-01-03-F-05-B,audio/PD_0364/PD_0364-15561-01-03-F-05-B.wav,transcription/PD_0364/PD_0364-15561-01-03-F-05...,오늘 점심으로 돈가스랑 마셨어요. 한 시 정도에요.,Female,30~39,대전/세종/충청/강원,충청,48000,Good
4,PD_0364-15396-01-03-F-05-B,audio/PD_0364/PD_0364-15396-01-03-F-05-B.wav,transcription/PD_0364/PD_0364-15396-01-03-F-05...,다이어트하느라 변비가 생겨서 요즘 화장실을 잘 못 가요.,Female,30~39,대전/세종/충청/강원,충청,48000,Good
...,...,...,...,...,...,...,...,...,...,...
309397,PF_0338-23233-01-03-M-04-B,audio/PF_0338/PF_0338-23233-01-03-M-04-B.wav,transcription/PF_0338/PF_0338-23233-01-03-M-04...,어느 쪽으로 눕혀야 되나요?,Male,20~29,대전/세종/충청/강원,충청,48000,Good
309398,PF_0338-23350-01-03-M-04-B,audio/PF_0338/PF_0338-23350-01-03-M-04-B.wav,transcription/PF_0338/PF_0338-23350-01-03-M-04...,몸무게가 오십 조금 넘으실 거예요. 키는 모르겠네요.,Male,20~29,대전/세종/충청/강원,충청,48000,Good
309399,PF_0338-23450-01-03-M-04-B,audio/PF_0338/PF_0338-23450-01-03-M-04-B.wav,transcription/PF_0338/PF_0338-23450-01-03-M-04...,자리를 지키고 있을 수가 없어서요. 꼭 문자 메시지 보내주세요.,Male,20~29,대전/세종/충청/강원,충청,48000,Good
309400,PF_0338-23563-01-03-M-04-B,audio/PF_0338/PF_0338-23563-01-03-M-04-B.wav,transcription/PF_0338/PF_0338-23563-01-03-M-04...,저는 다인실로 가지 않고 이인실에 쭉 있을 겁니다.,Male,20~29,대전/세종/충청/강원,충청,48000,Good


## 2-3. raw DataFrame 저장 및 불러오기

In [ ]:
# 인덱스 정리
filtered_df.reset_index(drop=True, inplace=True)

In [ ]:
# raw DataFrame 저장하기
filtered_df.to_csv(path + '/medical_audio_rawdata_2.csv', index=False)

In [ ]:
# len(df)

In [ ]:
# raw DataFrame 불러오기
df = pd.read_csv(path + 'medical_audio_rawdata_2.csv')

In [ ]:
# df['age'].value_counts()

In [ ]:
# df['labeltext'][:20]

In [ ]:
# df['gender'].value_counts()

In [ ]:
# 나이의 비율, 성별의 비
# df['region'].value_counts()

In [ ]:
# df['dialect'].value_counts()

# 3. 데이터 나누기(train, valid, test)

## 3-1. train 데이터 만들기

In [ ]:
# train 데이터프레임 만들기
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify = df['age'])

In [ ]:
# 비율 확인용
train_df['age'].value_counts()

In [ ]:
# 비율 확인용
test_df['age'].value_counts()

In [ ]:
# 테스트 인덱스 정리하기
test_df.reset_index(drop=True, inplace=True)

In [ ]:
test_df

,filename,audio_file_path,text_file_path,labeltext,gender,age,region,dialect,samplingrate,qualitystatus
0,PD_0172-13997-01-03-M-03-A,audio/PD_0172/PD_0172-13997-01-03-M-03-A.wav,transcription/PD_0172/PD_0172-13997-01-03-M-03...,눈앞이 빙글빙글 돌아요.,Male,11~19,서울/인천/경기,경기/서울,48000,Good
1,PE_0024-16282-01-03-M-06-A,audio/PE_0024/PE_0024-16282-01-03-M-06-A.wav,transcription/PE_0024/PE_0024-16282-01-03-M-06...,네. 여느 때랑 별 차이는 없어요.,Male,40~49,서울/인천/경기,경기/서울,48000,Good
2,PF_0066-20685-01-03-M-03-B,audio/PF_0066/PF_0066-20685-01-03-M-03-B.wav,transcription/PF_0066/PF_0066-20685-01-03-M-03...,잘 다녀왔어요.,Male,11~19,대전/세종/충청/강원,충청,48000,Good
3,PE_0051-16656-01-03-F-05-A,audio/PE_0051/PE_0051-16656-01-03-F-05-A.wav,transcription/PE_0051/PE_0051-16656-01-03-F-05...,네. 그냥 같았습니다.,Female,30~39,서울/인천/경기,경기/서울,48000,Good
4,PD_0246-14530-01-03-F-08-D,audio/PD_0246/PD_0246-14530-01-03-F-08-D.wav,transcription/PD_0246/PD_0246-14530-01-03-F-08...,별 감각이 없어요.,Female,60~69,부산/대구/울산/경상,경상,48000,Good
...,...,...,...,...,...,...,...,...,...,...
995,PD_0368-15554-01-03-M-07-B,audio/PD_0368/PD_0368-15554-01-03-M-07-B.wav,transcription/PD_0368/PD_0368-15554-01-03-M-07...,한 한 달 반은 지난 거 같아요.,Male,50~59,대전/세종/충청/강원,충청,48000,Good
996,PC_0221-10351-01-03-F-08-A,audio/PC_0221/PC_0221-10351-01-03-F-08-A.wav,transcription/PC_0221/PC_0221-10351-01-03-F-08...,투석 전문 병원인데 이름이 기억이 안 나요.,Female,60~69,서울/인천/경기,경기/서울,48000,Good
997,PC_0077-8531-01-03-F-08-A,audio/PC_0077/PC_0077-8531-01-03-F-08-A.wav,transcription/PC_0077/PC_0077-8531-01-03-F-08-...,좀 더 타면 안 되나요?,Female,60~69,서울/인천/경기,경기/서울,48000,Good
998,PD_0344-15350-01-03-M-04-B,audio/PD_0344/PD_0344-15350-01-03-M-04-B.wav,transcription/PD_0344/PD_0344-15350-01-03-M-04...,반 그릇 정도 먹을까 말까예요.,Male,20~29,대전/세종/충청/강원,충청,48000,Good


## 3-2. valid, test 데이터 만들기

In [ ]:
# valid, test 데이터프레임 만들기
valid_df, realtest_df = train_test_split(test_df, test_size=0.3, random_state=42, stratify = test_df['age'])

In [ ]:
# valid_df['dialect'].value_counts()

In [ ]:
# realtest_df['dialect'].value_counts()

## 3-3. 인덱스 정리 및 데이터프레임 저장하기

In [ ]:
# 인덱스 정리
train_df.reset_index(drop=True, inplace=True)
valid_df.reset_index(drop=True, inplace=True)
realtest_df.reset_index(drop=True, inplace=True)

In [ ]:
# 데이터프레임 저장하기
train_df.to_csv(path + '/medical_audio_train_2.csv', index=False)
valid_df.to_csv(path + '/medical_audio_valid_2.csv', index=False)
realtest_df.to_csv(path + '/medical_audio_test_2.csv', index=False)

In [ ]:
# train_df.shape

In [ ]:
# valid_df.shape

In [ ]:
# realtest_df.shape

# 4. 데이터 추출

In [ ]:
# 데이터프레임 불러오기
train_df = pd.read_csv(path + 'medical_audio_train_2.csv')
valid_df = pd.read_csv(path + 'medical_audio_valid_2.csv')
test_df = pd.read_csv(path + 'medical_audio_test_2.csv')

In [ ]:
# 데이터 선별 함수 만들기: 조건(사투리, 연령, 성별 비율)에 맞게 데이터 추출하기
def get_data(df, num):
    dialect_weights = df['dialect'].value_counts(normalize=True)
    age_weights = df['age'].value_counts(normalize=True)
    gender_weights = pd.Series({gender: 1/df['gender'].value_counts(normalize=True)[gender] for gender in df['gender'].unique()})
    sampling_weights = df.apply(lambda x: dialect_weights[x['dialect']] * age_weights[x['age']] * gender_weights[x['gender']], axis=1)

    # num개의 데이터 추출하기
    df = df.sample(n=num, weights=sampling_weights, replace=False)

    # 인덱스 정리하기
    df.reset_index(drop=True, inplace=True)
    return df

In [ ]:
# 함수를 써서 데이터 만들기
train_df = get_data(train_df, 10000)
valid_df = get_data(valid_df, 5000)
test_df = get_data(test_df, 1000)

In [ ]:
train_df['dialect'].value_counts()

dialect
경기/서울    4279
경상       3049
충청       1405
전라       1256
강원         11
Name: count, dtype: int64

In [ ]:
valid_df['dialect'].value_counts()

dialect
경기/서울     2120
경상        1486
충청         759
전라         629
강원           5
기타(외국)       1
Name: count, dtype: int64

In [ ]:
test_df['dialect'].value_counts()

dialect
경기/서울    427
경상       312
충청       144
전라       117
Name: count, dtype: int64

In [ ]:
# 최종 학습 데이터 저장
train_df.to_csv(path + 'pre_medical_audio_train.csv', index=False)
valid_df.to_csv(path + 'pre_medical_audio_valid.csv', index=False)
test_df.to_csv(path + 'pre_medical_audio_test.csv', index=False)